In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)

from qiskit_braket_provider import BraketProvider
from qiskit.providers import Backend

provider = BraketProvider()
aria_2 : Backend = provider.get_backend("Aria 2")

from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit

hamiltonian: FermionicOp = (
    PySCFDriver(
        atom="H 0 0 0; H 0 0 0.735",
        basis="sto3g",
        charge=0,
        spin=0,
        unit=DistanceUnit.ANGSTROM,
    )
    .run()
    .hamiltonian.second_q_op()
)

In [3]:
from qiskit import transpile
from qiskit_nature.second_q.mappers import JordanWignerMapper, BravyiKitaevMapper
from utility import Execution

jwsim = Execution(hamiltonian, JordanWignerMapper())

Hamiltonian simulation:
  Solve ground state:
Statevector([-2.32088474e-17-1.32768052e-17j,
             -1.29593802e-16+2.54992228e-17j,
              1.09129429e-19+7.39911913e-18j,
              2.50876051e-16-8.50052294e-17j,
              6.31998588e-17+1.76362353e-16j,
             -7.06503802e-01-6.98864874e-01j,
             -8.62676944e-17-3.36979993e-17j,
              6.60724127e-18+2.58509894e-17j,
              1.09888642e-17+8.45645702e-18j,
             -8.88352769e-17-1.64176064e-16j,
              7.92953359e-02+7.84379713e-02j,
             -1.15845698e-17-2.30657371e-17j,
              1.33221826e-16-4.94907237e-17j,
             -7.40020536e-18-8.88351305e-18j,
             -8.23486807e-19-1.63446144e-17j,
              1.32576421e-16+1.16797329e-16j],
            dims=(2, 2, 2, 2))
    Ground energy = -1.8572750302023833
  Construct and transpile Pauli evolution:
    Time duration    = 1.0
    Basis gates      = cx, u3
    Synthesis method = LieTrotter
  Circuit su

In [3]:
bk_job = aria_2.run(bksim.circuit, shots=3000)

In [8]:
bk_job.result().data()["counts"]

{'0000': 245,
 '0001': 143,
 '0010': 798,
 '0011': 232,
 '0100': 266,
 '0101': 120,
 '0110': 155,
 '0111': 457,
 '1000': 121,
 '1001': 52,
 '1010': 109,
 '1011': 87,
 '1100': 44,
 '1101': 45,
 '1110': 59,
 '1111': 67}

In [15]:
from qiskit.quantum_info import Statevector
from qiskit_nature.second_q.mappers.fermionic_mapper import FermionicMapper
from statistics import mean, variance

def process(data: dict[str, int], mapper: FermionicMapper, hamiltonian):
    hamiltonian = mapper.map(hamiltonian)

    energies = []

    for state, n in data.items():
        initial_state = Statevector.from_label(state[::-1])
        exp_energy = initial_state.expectation_value(hamiltonian)  # type: ignore
        print(exp_energy, n)

        for _ in range(n):
            energies.append(exp_energy.real)

    return (
        mean(energies),
        variance(energies),
    )

In [16]:
process(bk_job.result().data()["counts"], BravyiKitaevMapper(), hamiltonian)

(1.1102230246251565e-16+0j) 245
(-0.4718960072811417+0j) 143
(-1.2445845498133268+0j) 798
(-1.25633907300325+0j) 232
(-0.24521829183026578+0j) 266
(-0.4718960072811416+0j) 120
(-1.0636533500290954+0j) 155
(-0.35332510410715445+0j) 457
(-1.2445845498133268+0j) 121
(-0.35332510410715456+0j) 52
(0.21427823841947297+0j) 109
(-1.1606317377577644+0j) 87
(-1.0636533500290954+0j) 44
(-1.2563390730032498+0j) 45
(-1.8369679912029837+0j) 59
(-1.1606317377577642+0j) 67


(-0.7787958019321397, 0.2720765019864728)